In [1]:
import os
import logging
import pickle

import xarray as xr

from config import CACHE, RESULTS_PATH, setup_logging
from demo.model_configs import analysis_cfg as cfg
from code.model_activations.models.utils import load_model, load_full_identifier
from code.model_activations.activation_extractor import Activations
from code.image_classification.classification_tools import get_Xy, cv_performance
from code.eigen_analysis.compute_pcs import compute_model_pcs

setup_logging()

/home/atlask/Desktop/untrained_models_of_visual_cortex/code/model_activations/activation_extractor.py:21: UserWarning: my warning
  warnings.warn('my warning')


In [2]:
MODEL_NAMES = ['expansion','alexnet']
PCA_DATASET = 'places_train_demo'
DATASET = 'places_val_demo'
NUM_COMPONENTS = 100
DEVICE = 'cuda'
BATCH_SIZE = 50


In [6]:
for model_name in MODEL_NAMES:
    # get model activation iden
    activations_identifier = load_full_identifier(model_name=model_name, 
                                            features=cfg[DATASET]['models'][model_name]['features'], 
                                            layers=cfg[DATASET]['models'][model_name]['layers'], 
                                            dataset=DATASET,
                                            principal_components = NUM_COMPONENTS)
    
    if not os.path.exists(os.path.join(RESULTS_PATH,f'classification_{model_name}')):
        print(f"Getting classification results for: {activations_identifier}")
        
        
        pca_identifier = load_full_identifier(model_name=model_name, 
                                                    features=cfg[PCA_DATASET]['models'][model_name]['features'], 
                                                    layers=cfg[PCA_DATASET]['models'][model_name]['layers'], 
                                                    dataset=PCA_DATASET,
                                                    principal_components = NUM_COMPONENTS)
        
        print(f"Computing model principal components using a subset of the Places train set")
        if not os.path.exists(os.path.join(CACHE,'pca',pca_identifier)):
            compute_model_pcs(model_name = model_name, 
                            features=cfg[PCA_DATASET]['models'][model_name]['features'],  
                            layers=cfg[PCA_DATASET]['models'][model_name]['layers'], 
                            dataset = PCA_DATASET, 
                            components = NUM_COMPONENTS, 
                            device = DEVICE,
                            batch_size=BATCH_SIZE)
            
        # load model
        model = load_model(model_name=model_name, 
                        features=cfg[DATASET]['models'][model_name]['features'], 
                        layers=cfg[DATASET]['models'][model_name]['layers'],
                        device=DEVICE)
        
        print(f"Extracting activations from the Places val set and projecting them onto the learned PCs")
        # compute activations and project onto PCs
        Activations(model=model, 
                    dataset=DATASET, 
                    pca_iden = pca_identifier,
                    n_components = NUM_COMPONENTS, 
                    batch_size = BATCH_SIZE,
                    device= DEVICE).get_array(activations_identifier)  
        
        logging.info(f"Extracting activations from the Places val set and projecting them onto the learned PCs")
        X, y = get_Xy(data=xr.open_dataset(os.path.join(CACHE,'activations',activations_identifier), 
                            engine='netcdf4').set_xindex('stimulus_id'))
        score = cv_performance(X, y, class_balance=False)

        with open(os.path.join(RESULTS_PATH,f'classification_{model_name}'),'wb') as f:
            pickle.dump(score,f)
    else:
        print(f'results for model: {activations_identifier} are already saved in cache')

Getting classification results for: expansion_features=3_layers=5_dataset=places_val_demo_principal_components=100
Computing model principal components using a subset of the Places train set


2024-06-27 14:55:20,806 - INFO - Loading processed images...
2024-06-27 14:55:20,994 - INFO - Extracting activations...


Extracting activations from the Places val set and projecting them onto the learned PCs


100%|██████████| 10/10 [00:26<00:00,  2.60s/it]
2024-06-27 14:55:47,128 - INFO - Model activations are saved in cache
2024-06-27 14:55:47,214 - INFO - Extracting activations from the Places val set and projecting them onto the learned PCs


Getting classification results for: alexnet_conv5_layers=5_features=256_dataset=places_val_demo_principal_components=100
Computing model principal components using a subset of the Places train set


/home/atlask/anaconda3/envs/final_test/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/atlask/anaconda3/envs/final_test/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
2024-06-27 14:55:54,731 - INFO - Loading processed images...


Extracting activations from the Places val set and projecting them onto the learned PCs


2024-06-27 14:55:54,929 - INFO - Extracting activations...
100%|██████████| 10/10 [00:00<00:00, 59.59it/s]
2024-06-27 14:55:55,101 - INFO - Model activations are saved in cache
2024-06-27 14:55:55,182 - INFO - Extracting activations from the Places val set and projecting them onto the learned PCs


In [9]:
# load expansion results
for model_name in MODEL_NAMES:

    with open(os.path.join(RESULTS_PATH, f'classification_{model_name}'),'rb') as f:
        accuracy_score = pickle.load(f)
    print(model_name, ':', accuracy_score)

expansion : 0.002
alexnet : 0.048
